In [25]:
from openbabel import openbabel

In [26]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [28]:
from treelib import Node, Tree

In [29]:
class Molecule(object):
    def __init__(self, path):
        self.path = path

In [30]:
def split(word):
    return[char for char in word]

In [60]:
def index_finder(Product, rels, path):
    indexes = []
    energies = []
    for i in range(len(rels['Reagent'])):
        if rels['Reagent'][i] == Product and rels['Formed/Produced'][i] == 1:
            indexes.append(rels['Index'][i])
            energies.append(rels['Energy Change'][i])
    final_index = 0
    final_energy = 10e10
    for i in range(len(indexes)):
        if energies[i] < final_energy:
            valid = True
            precursors = precursor_finder(indexes[i], rels)
            for j in range(len(precursors)):
                if precursors[j] in path:
                    valid = False
                    break
            if valid == True:
                final_index = indexes[i]
                final_energy = energies[i] 
    return(final_index)

In [61]:
def precursor_finder(index, rels):
    precursors = []
    for i in range(len(rels['Index'])):
        if rels['Index'][i] == index and rels['Formed/Produced'][i] == -1:
            precursors.append(rels['Reagent'][i])
    return(precursors)

In [62]:
def find_component(node, rels):
    Product = node.tag
    path = split((node.data).path)
    index = index_finder(Product, rels, path)
    precursors = precursor_finder(index, rels)
    return(precursors)

In [63]:
def map_tree(Smiles, rels):
    rels = pd.read_csv(rels, sep='\t')
    #base_molecules = ['A', 'B', 'C']
    base_molecules = ['N', 'C=O', 'C(CO)=O', 'O']
    tree = Tree()
    tree.create_node(Smiles, 0, data=Molecule(Smiles))
    
    finished = False
    level_counter = 0
    node_counter = 0
    while finished == False:
        nodes = tree.all_nodes()
        active_nodes = []
        for i in range(len(nodes)):
            tag = nodes[i].tag
            level = tree.depth(nodes[i])
            if level == level_counter and tag not in base_molecules:
                active_nodes.append(nodes[i])
        if active_nodes == []:
            finished = True
            break
        else:
            for i in range(len(active_nodes)):
                precursors = find_component(active_nodes[i], rels)
                product = active_nodes[i].identifier
                dummy = (active_nodes[i].data).path
                for i in range(len(precursors)):
                    node_counter +=1
                    uniquepath = dummy + precursors[i]
                    tree.create_node(precursors[i], node_counter, parent=product, data=Molecule(uniquepath))
            level_counter+=1
    return(tree)

In [64]:
a = map_tree('H', './Test_rels.tsv')

In [65]:
a.show()

H
└── A



In [66]:
a.show(data_property='path')

H
└── HA

